# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 13 2022 2:56PM,253026,19,MSP216752,"Methapharm, Inc.",Released
1,Dec 13 2022 2:46PM,253025,19,ACG-2102492147,ACG North America LLC,Released
2,Dec 13 2022 2:14PM,253023,10,0086245946,ISDIN Corporation,Executing
3,Dec 13 2022 2:14PM,253023,10,0086245947,ISDIN Corporation,Executing
4,Dec 13 2022 2:14PM,253023,10,0086245949,ISDIN Corporation,Executing
5,Dec 13 2022 2:14PM,253023,10,0086245950,ISDIN Corporation,Executing
6,Dec 13 2022 2:14PM,253023,10,0086245948,ISDIN Corporation,Executing
7,Dec 13 2022 2:14PM,253023,10,0086245953,ISDIN Corporation,Executing
8,Dec 13 2022 2:14PM,253023,10,0086245956,ISDIN Corporation,Executing
9,Dec 13 2022 2:14PM,253023,10,0086245960,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,253019,Executing,1
46,253023,Executing,10
47,253023,Released,8
48,253025,Released,1
49,253026,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253018,NaN,3.0,1.0
253019,NaN,1.0,NaN
253023,NaN,10.0,8.0
253025,NaN,NaN,1.0
253026,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252899,0.0,0.0,1.0
252932,24.0,14.0,42.0
252933,42.0,20.0,1.0
252934,13.0,0.0,7.0
252947,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252899,0,0,1
252932,24,14,42
252933,42,20,1
252934,13,0,7
252947,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252899,0,0,1
1,252932,24,14,42
2,252933,42,20,1
3,252934,13,0,7
4,252947,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252899,,,1
1,252932,24,14,42
2,252933,42,20,1
3,252934,13,,7
4,252947,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc."
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation
20,Dec 13 2022 1:59PM,252994,10,Bio-PRF
21,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc."
22,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix
26,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc."
27,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc."
28,Dec 13 2022 1:27PM,253014,10,Emerginnova
30,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc.",,,1
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,,,1
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation,,10,8
3,Dec 13 2022 1:59PM,252994,10,Bio-PRF,,,1
4,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",,1,
5,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,,3,1
6,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",,,1
7,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",,,1
8,Dec 13 2022 1:27PM,253014,10,Emerginnova,,,2
9,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc.",1,,
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,1,,
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation,8,10,
3,Dec 13 2022 1:59PM,252994,10,Bio-PRF,1,,
4,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",,1,
5,Dec 13 2022 1:46PM,253018,10,Yusen – 3D Matrix,1,3,
6,Dec 13 2022 1:45PM,253017,19,"GUSA Granules USA, Inc.",1,,
7,Dec 13 2022 1:29PM,253015,19,"GUSA Granules USA, Inc.",1,,
8,Dec 13 2022 1:27PM,253014,10,Emerginnova,2,,
9,Dec 13 2022 1:25PM,253012,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc.",1,,
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,1,,
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation,8,10,
3,Dec 13 2022 1:59PM,252994,10,Bio-PRF,1,,
4,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc.",1.0,NaN,NaN
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,1.0,NaN,NaN
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation,8.0,10.0,NaN
3,Dec 13 2022 1:59PM,252994,10,Bio-PRF,1.0,NaN,NaN
4,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 13 2022 2:56PM,253026,19,"Methapharm, Inc.",1.0,0.0,0.0
1,Dec 13 2022 2:46PM,253025,19,ACG North America LLC,1.0,0.0,0.0
2,Dec 13 2022 2:14PM,253023,10,ISDIN Corporation,8.0,10.0,0.0
3,Dec 13 2022 1:59PM,252994,10,Bio-PRF,1.0,0.0,0.0
4,Dec 13 2022 1:52PM,253019,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5818848,91.0,43.0,0.0
12,252958,1.0,0.0,0.0
15,758799,50.0,34.0,79.0
16,758851,3.0,0.0,0.0
17,252964,1.0,0.0,0.0
19,1012083,4.0,0.0,0.0
20,505947,2.0,0.0,0.0
21,758990,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5818848,91.0,43.0,0.0
1,12,252958,1.0,0.0,0.0
2,15,758799,50.0,34.0,79.0
3,16,758851,3.0,0.0,0.0
4,17,252964,1.0,0.0,0.0
5,19,1012083,4.0,0.0,0.0
6,20,505947,2.0,0.0,0.0
7,21,758990,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,91.0,43.0,0.0
1,12,1.0,0.0,0.0
2,15,50.0,34.0,79.0
3,16,3.0,0.0,0.0
4,17,1.0,0.0,0.0
5,19,4.0,0.0,0.0
6,20,2.0,0.0,0.0
7,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,91.0
1,12,Released,1.0
2,15,Released,50.0
3,16,Released,3.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,79.0,0.0,0.0,0.0,0.0,0.0
Executing,43.0,0.0,34.0,0.0,0.0,0.0,0.0,2.0
Released,91.0,1.0,50.0,3.0,1.0,4.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,79.0,0.0,0.0,0.0,0.0,0.0
1,Executing,43.0,0.0,34.0,0.0,0.0,0.0,0.0,2.0
2,Released,91.0,1.0,50.0,3.0,1.0,4.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,79.0,0.0,0.0,0.0,0.0,0.0
1,Executing,43.0,0.0,34.0,0.0,0.0,0.0,0.0,2.0
2,Released,91.0,1.0,50.0,3.0,1.0,4.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()